In [1]:
# config
import sys
import pandas as pd
!pip3 install praw
import praw
import time
sys.path.append('../') 
import config.settings as settings
from pathlib import Path

reddit = praw.Reddit(
    client_id=settings.PRAW_CLIENT,
    client_secret=settings.PRAW_CLIENT_SECRET,
    user_agent=settings.PRAW_USER_AGENT
)

sleep_time = 2

## Constructing Table Structure and Calling Reddit API
Post limit sets to 500

In [ ]:
post_title = []
post_id = []
post_datetime = []
post_score = []
post_body = []
comment_id = []
comment_text = []
comment_datetime = []
comment_score = []
seen_ids = set()

In [ ]:
total_fetched = 0
target_posts = 900
start_time = time.time()
posts = reddit.subreddit("Taiwan").new()

for post in posts:
        if post.id not in seen_ids and post.selftext:
            print(f"crawling comments from post {post.title}")
            seen_ids.add(post.id)
            total_fetched += 1
            post_title.append(post.title)
            post_id.append(post.id)
            post_datetime.append(post.created_utc)
            post_score.append(post.score)
            post_body.append(post.selftext)

In [ ]:
target_posts = 5
start_time = time.time()
posts = reddit.subreddit("Hongkong").random_rising()
total_fetched=0
while total_fetched<=target_posts:
    total_fetched += 1
    for post in posts:
        if post.id not in seen_ids and post.selftext:
            print(f"crawling from post {post.title}")
            seen_ids.add(post.id)
            post_title.append(post.title)
            post_id.append(post.id)
            post_datetime.append(post.created_utc)
            post_score.append(post.score)
            post_body.append(post.selftext)
        if total_fetched >= target_posts: break

In [ ]:
posts = reddit.subreddit("China").rising()
for post in posts:
    print(post.title)

## Export to CSV

In [ ]:
import os
from pathlib import Path

raw_data = {
    "post_title": post_title,
    "post_id": post_id,
    "post_body": post_body,
    "post_datetime": post_datetime,
    "post_score": post_score,
}

analysis_path = Path.joinpath(settings.PROJ_PATH,"analysis_data")

raw_data_df = pd.DataFrame(raw_data).drop_duplicates(subset=["post_id"],keep="first")
raw_data_df.to_csv(Path.joinpath(analysis_path,"/raw_data_hongkong_post_1.csv"),index=False)

## Crawling comments given postID

In [ ]:
import os

def fetch_comments_for_posts(reddit, df):
    comment_data = []

    for _, row in df.iterrows():
        post_id = row['post_id']
        post_title = row['post_title']
        post_body = row['post_body']
        post_datetime = row['post_datetime']
        post_score = row['post_score']

        submission = reddit.submission(id=post_id)

        # Load comments
        submission.comments.replace_more(limit=0)  # To ensure all top-level comments are loaded
        for comment in submission.comments:
            comment_data.append({
                'post_title': post_title,
                'post_id': post_id,
                'post_body': post_body,
                'post_datetime': post_datetime,
                'post_score': post_score,
                'post_owner': submission.author.name if submission.author else None,
                'comment_owner': comment.author.name if comment.author else None,
                'reply_to_userId': comment.parent().author.name if comment.parent().author else None,
                'comment_datetime': comment.created_utc,
                'comment_score': comment.score
            })
    
    return pd.DataFrame(comment_data)

def process_files(data_path, output_path, reddit):
    for file in os.listdir(data_path):
        if file.endswith('.csv'):
            file_path = os.path.join(data_path, file)
            df = pd.read_csv(file_path)
            
            comments_df = fetch_comments_for_posts(reddit, df)
            output_file = os.path.join(output_path, f"{file.replace('raw_data_', '').replace('_post.csv', '')}_comments.csv")
            comments_df.to_csv(output_file, index=False)


In [ ]:
post_data_dir = Path.joinpath(settings.PROJ_PATH,"post_data")
process_files(post_data_dir, post_data_dir, reddit)